# Define Regions of Interest (ROIs)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle as pkl
from pathlib import Path

import napari
from plex_pipe.widgets import RoiWidget
from plex_pipe import load_config

## Read in config

In [2]:
# load analysis configuration
config_path = r'../examples/example_pipeline_config.yaml'

config = load_config(config_path)

2026-02-20 10:47:23.201 | WARNING  | plex_pipe.processors.base:_log_missing_optional_params:49 - InstansegSegmenter.Params: Parameter overlap not provided. Using default value: 80.


## Create a Napari viewer and define ROIs manually

In [3]:
viewer = napari.Viewer()
roi_widget = RoiWidget.from_config(viewer, config)
viewer.window.add_dock_widget(roi_widget, area='left')

---
---

# Get ROIs suggested by the SAM2 model

Execute the cells below to get suggestions from SAM model about the cores position. 

Sometimes there is no immediate feedback of how SAM2 progresses. In this situation it's expected to wait 1-2 min for the outcome.

In [4]:
import subprocess
import platform
import numpy as np

from plex_pipe.utils.config_loaders import load_workstation_config

from plex_pipe.utils.file_utils import (
    change_to_wsl_path
)
from plex_pipe.core_definition.roi_utils import (
    get_refined_rectangles,
    prepare_poly_df_for_saving,
    get_visual_rectangles,
)

from plex_pipe.widgets.viewer_utils import (
    redo_bbox_layer,
    redo_cores_layer
)

In [14]:
# get necessary variables from the widget and config
im_level = roi_widget.im_level
im_list = roi_widget.im_list
edge_width = roi_widget.edge_width
org_im_shape = roi_widget.org_im_shape
image_path = Path(config.general.image_dir) / config.roi_definition.detection_image

In [10]:
# load pipeline configuration
sam_config = load_workstation_config(r'../examples/example_sam_config.yaml')
sam_config

{'model_path': '/mnt/d/sam2_test/sam2',
 'sam_env': '/home/kasia/miniforge3/envs/sam2-env/bin/python'}

## Define sampling

In [11]:
#########################################################################################
nominal_small_size = 80  #the "ideal" size of the smallest objects you expect.
min_delta_factor = 0.75 # the multiplier to find the absolute minimum (e.g., 0.5×nominal).
max_delta_factor = 1.25 # the multiplier to find the absolute maximum (e.g., 2.0×nominal)."
#########################################################################################

n_sampling_points = 2 * int((np.max(im_list[0].shape) / (nominal_small_size)))
print(f'For objects of size: {nominal_small_size}, number of points per side for SAM2 segmentation will be: {n_sampling_points}')

For objects of size: 80, number of points per side for SAM2 segmentation will be: 4


## SAM2 Segmentation

In [12]:
# read image stretching parameters from the viewer
clims = viewer.layers['signal'].contrast_limits
display_level = viewer.layers['signal']._data_level
res_level = int(display_level + im_level)# if im_level is not zero than display level will be zero

# determine the operating system
is_windows = platform.system() == "Windows"

# handle paths
im_path = str(image_path.resolve())
out_path = os.path.abspath('masks.pkl')

if is_windows:
    im_path_wsl = change_to_wsl_path(str(image_path.resolve()))
    out_path_wsl = change_to_wsl_path(os.path.join(os.getcwd(), 'masks.pkl'))

# define arguments
args = [
    "--image_path", im_path_wsl if is_windows else im_path,
    "--req_level", str(res_level),
    "--int_min", str(clims[0]),
    "--int_max", str(clims[1]),
    "--model_path", sam_config['model_path'],
    "--output_file", out_path_wsl if is_windows else out_path,
    "--points_per_side", str(n_sampling_points)
]

# define subprocess command
if is_windows:
    command = ["wsl", sam_config['sam_env'], "-m", "plex_pipe.core_definition.suggest_cores"] + args
else:
    python_exe = sam_config.get('sam_env', 'python')
    command = [python_exe, "-m", "plex_pipe.core_definition.suggest_cores"] + args

# run execution
process = subprocess.Popen(
    command,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

# process subprocess messages and errors
while True:
    output = process.stdout.readline()
    if output == "" and process.poll() is not None:
        break
    if output:
        print(output, end="")

errors = process.stderr.read()
if errors:
    print(f"ERROR: {errors}")

Input image: /mnt/d/plex-pipe/examples/input/sample_1.0.4_R000_DAPI__FINAL_F.tiff
Results will be saved to: /mnt/d/plex-pipe/notebooks/masks.pkl

Preparing RGB image for segmentation...
Segmenting image. It should take around 1 min. Started at 2026-02-20 10:44:25...
Saving masks...
ERROR: /home/kasia/miniforge3/envs/sam2-env/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/kasia/miniforge3/envs/sam2-env/lib/python3.11/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.


## Filter and display masks

In [15]:
# masks processing & display

# load the masks
masks = pkl.load(open(out_path, 'rb'))
print(f'Initial number of masks: {len(masks)}')

# refine masks
rect_list = get_refined_rectangles(masks, im = im_list[display_level],
                                   min_area = (min_delta_factor * nominal_small_size/(2**display_level))**2,
                                   max_area = (max_delta_factor * nominal_small_size/(2**display_level))**2,
                                   min_iou = 0.8,
                                   min_stability = 0.9,
                                   min_int = np.mean(im_list[display_level]) # only masks of objects of intensity above this threshold are retained
                                   )
print('Number of refined masks:', len(rect_list))

# prepare df
df = prepare_poly_df_for_saving(rect_list, poly_types = ['rectangle']*len(rect_list), org_im_shape = org_im_shape, req_level = res_level)
rect_list = get_visual_rectangles(df, im_level)
poly_list = [(x/(2**im_level)).astype('int') for x in df.polygon_vertices.to_list()]

# add to napari
redo_bbox_layer(viewer,rect_list,df['roi_name'].tolist(), edge_width = edge_width)
redo_cores_layer(viewer,rect_list,shape_type = df.poly_type.to_list(), edge_width = edge_width)

# make cores layer active
viewer.layers.selection.clear()
viewer.layers.selection.add(viewer.layers['ROIs'])

Initial number of masks: 3
Number of refined masks: 2


In [16]:
# cleanup intermediate SAM2 result
os.remove(out_path)

In [17]:
import pandas as pd
pd.read_pickle('../examples/output/sample_analysis/rois.pkl')

,roi_name,row_start,row_stop,column_start,column_stop,poly_type,polygon_vertices
0,ROI_000,64.0,5056.0,256.0,5248.0,rectangle,"[[64.0, 256.0], [64.0, 5248.0], [5056.0, 5248...."
1,ROI_001,128.0,4992.0,6912.0,11776.0,rectangle,"[[128.0, 6912.0], [128.0, 11776.0], [4992.0, 1..."
